분석에 이용한 자료는 2020-03-11 22:00에 수집하였습니다.

In [3]:
import pickle
import json
import requests
import pandas as pd
import numpy as np
import chart_studio

import plotly
import plotly.graph_objects as go

from pandas import DataFrame
from scipy import stats

In [4]:
plotly.offline.init_notebook_mode(connected=True)

In [5]:
with open('data.pickle', 'rb') as f:
    fifa = pickle.load(f)

In [6]:
spId_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spid.json')
spId_parsed_data = spId_url.json()
spId = pd.DataFrame(spId_parsed_data)

In [7]:
spposition_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spposition.json')
spposition_parsed_data = spposition_url.json()
spposition = pd.DataFrame(spposition_parsed_data)

- ```assist``` : 평균 어시스트 수
- ```blcok``` : 평균 블락 성공 수
- ```dribble``` : 평균 드리블 거리(야드)
- ```effectiveShoot``` : 평균 유효 슛 수
- ```goal``` : 평균 득점 수
- ```matchCount``` : 해당 포지션으로 경기 참여한 횟수
- ```passSuccess``` : 평균 패스 성공 수
- ```tackle``` : 평균 태클 성공 수

In [29]:
fifa.head()

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,101000001,0,0.0,0.0,0.0,0.0,0.0,4,0.0,0.928571
1,101000001,28,0.0,0.0,0.0,0.0,0.0,2,0.0,NaN
2,101000195,4,0.0,0.0,0.0,0.0,0.0,1,0.0,1.000000
3,101000195,12,0.5,0.0,85.0,2.0,0.0,2,0.0,0.848485
4,101000195,14,2.0,0.0,0.0,0.0,0.0,1,0.0,0.928571


### 골키퍼(GK)

In [8]:
gk = fifa[(fifa.spPosition == 0) & (fifa.matchCount == 20)]
gk_top3 = gk.sort_values(by = 'passRate', ascending = False).head(3)
gk_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
43629,218199987,0,0.0,0.0,0.0,0.0,0.0,20,0.0,0.984375
34174,215215223,0,0.0,0.0,0.0,0.0,0.0,20,0.0,0.981481
123009,502199987,0,0.0,0.0,0.0,0.0,0.0,20,0.0,0.977273


패스 성공률 상위 3선수의 패스 성공률

In [34]:
gk_passRate = list(gk_top3['passRate'])

패스 성공률 상위 10선수의 이름

In [35]:
gk_topId = []

for i in gk_top3['spId'] :
    gk_topId.append(i)

gk_topSp = []

for j in gk_topId :
    gk_topSp.append(list(spId[spId['id'] == j].name))

gk_topSp = sum(gk_topSp, [])
gk_topSp

['J. 실러선', 'W. 베니테스', 'J. 실러선']

In [36]:
fig = go.Figure([go.Bar(x=gk_topSp, y=gk_passRate)])
fig.show()

오각형 모양의 선수 스탯 그래프 그리기 연습

In [32]:
test_gk = gk_top3.head(1)

In [38]:
data = [go.Scatterpolar(
    r = [test_gk['assist'].values[0],test_gk['block'].values[0], test_gk['effectiveShoot'].values[0],
         test_gk['passRate'].values[0],test_gk['tackle'].values[0]],
    
    theta = ['assist', 'block', 'effectiveShoot', 'passRate', 'tackle'],

    fill = 'toself')]


fig = go.Figure(data = data)
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()

################################################

In [67]:
categories = ['assist','block','effectiveShoot','tackle']

## 공격수

### 스트라이커(ST)

In [68]:
st = fifa[(fifa.spPosition == 25) & (fifa.matchCount == 20)]
st_top3 = st.sort_values(by = 'effectiveShoot', ascending = False).head(3)
st_top3.reset_index( inplace = True )
st_top3.drop(['index'], axis=1, inplace = True)
st_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,101031432,25,0.15,0.0,25.15,3.95,1.90,20,0.15,0.875000
1,207000051,25,0.05,0.0,45.85,3.05,0.95,20,0.15,0.965714
2,215224179,25,0.20,0.0,54.35,2.30,1.15,20,0.25,0.922414


In [115]:
st_topSp = []

for i in st_top3['spId'] :
    st_topSp.append(list(spId[spId['id'] == i].name))
    
st_topSp = sum(st_topSp, [])

In [116]:
fig = go.Figure()



fig.add_trace(go.Scatterpolar(
      r=[st_top3.loc[0]['assist'], st_top3.loc[0]['block'], 
         st_top3.loc[0]['effectiveShoot'], st_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=st_topSp[0]
))

fig.add_trace(go.Scatterpolar(
      r=[st_top3.loc[1]['assist'], st_top3.loc[1]['block'], 
         st_top3.loc[1]['effectiveShoot'], st_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=st_topSp[1]
))

fig.add_trace(go.Scatterpolar(
      r=[st_top3.loc[2]['assist'], st_top3.loc[2]['block'], 
         st_top3.loc[2]['effectiveShoot'], st_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=st_topSp[2]
))



fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 4.5]
    )),
  showlegend=True,
    title = {
        'text': '주요 스트라이커 능력치 비교',
        'x': 0.45, 
        'y': 0.9,
    }
)

fig.show()

### 센터 포워드(CF)

In [64]:
cf = fifa[(fifa.spPosition == 21) & (fifa.matchCount == 20)]
cf_top3 = cf.sort_values(by = 'effectiveShoot', ascending = False).head(3)
cf_top3.reset_index( inplace = True )
cf_top3.drop(['index'], axis=1, inplace = True)
cf_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,101190045,21,0.40,0.05,32.75,2.20,0.85,20,0.25,0.845921
1,219242444,21,0.20,0.00,3.95,2.15,0.45,20,0.10,0.884868
2,295191910,21,0.35,0.00,34.85,1.85,0.85,20,0.35,0.865031


In [117]:
cf_topSp = []

for i in cf_top3['spId'] :
    cf_topSp.append(list(spId[spId['id'] == i].name))
    
cf_topSp = sum(cf_topSp, [])

In [128]:
fig = go.Figure()



fig.add_trace(go.Scatterpolar(
      r=[cf_top3.loc[0]['assist'], cf_top3.loc[0]['block'], 
         cf_top3.loc[0]['effectiveShoot'], cf_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=cf_topSp[0]
))

fig.add_trace(go.Scatterpolar(
      r=[cf_top3.loc[1]['assist'], cf_top3.loc[1]['block'], 
         cf_top3.loc[1]['effectiveShoot'], cf_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=cf_topSp[1]
))

fig.add_trace(go.Scatterpolar(
      r=[cf_top3.loc[2]['assist'], cf_top3.loc[2]['block'], 
         cf_top3.loc[2]['effectiveShoot'], cf_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=cf_topSp[2]
))



fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 2.5]
    )),
  showlegend=True,
    title = {
        'text': '주요 센터 포워드 능력치 비교',
        'x': 0.46, 
        'y': 0.9
    }
)

fig.show()

### 중앙 미드필더(CM)

In [136]:
cm = fifa[(fifa.spPosition == 14) & (fifa.matchCount == 20)]
cm_top3 = cm.sort_values(by = 'passRate', ascending = False).head(3)
cm_top3.reset_index( inplace = True )
cm_top3.drop(['index'], axis=1, inplace = True)
cm_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,206233419,14,0.2,0.05,54.50,0.60,0.15,20,0.45,0.952218
1,207177003,14,0.0,0.00,3.10,0.35,0.05,20,0.40,0.947552
2,221156519,14,0.3,0.00,27.55,0.60,0.25,20,0.95,0.945161


In [137]:
cm_topSp = []

for i in cm_top3['spId'] :
    cm_topSp.append(list(spId[spId['id'] == i].name))
    
cm_topSp = sum(cm_topSp, [])

In [142]:
fig = go.Figure()



fig.add_trace(go.Scatterpolar(
      r=[cm_top3.loc[0]['assist'], cm_top3.loc[0]['block'], 
         cm_top3.loc[0]['effectiveShoot'], cm_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=cm_topSp[0]
))

fig.add_trace(go.Scatterpolar(
      r=[cm_top3.loc[1]['assist'], cm_top3.loc[1]['block'], 
         cm_top3.loc[1]['effectiveShoot'], cm_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=cm_topSp[1]
))

fig.add_trace(go.Scatterpolar(
      r=[cm_top3.loc[2]['assist'], cm_top3.loc[2]['block'], 
         cm_top3.loc[2]['effectiveShoot'], cm_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=cm_topSp[2]
))



fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1.2]
    )),
  showlegend=True,
    title = {
        'text': '주요 중앙 미드필더 능력치 비교',
        'x': 0.46, 
        'y': 0.9
    }
)

fig.show()

## 수비수

### 윙어(LW)

In [148]:
lw = fifa[(fifa.spPosition == 27) & (fifa.matchCount == 20)]
lw_top3 = lw.sort_values(by = 'passRate', ascending = False).head(3)
lw_top3.reset_index( inplace = True )
lw_top3.drop(['index'], axis=1, inplace = True)
lw_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,210189596,27,0.20,0.0,28.75,0.30,0.20,20,0.15,0.941176
1,221208722,27,0.20,0.0,35.75,0.40,0.20,20,0.20,0.936047
2,206184267,27,0.25,0.0,69.05,0.65,0.15,20,0.20,0.931624


In [149]:
lw_topSp = []

for i in lw_top3['spId'] :
    lw_topSp.append(list(spId[spId['id'] == i].name))
    
lw_topSp = sum(lw_topSp, [])

In [157]:
fig = go.Figure()



fig.add_trace(go.Scatterpolar(
      r=[lw_top3.loc[0]['assist'], lw_top3.loc[0]['block'], 
         lw_top3.loc[0]['effectiveShoot'], lw_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=lw_topSp[0]
))

fig.add_trace(go.Scatterpolar(
      r=[lw_top3.loc[1]['assist'], lw_top3.loc[1]['block'], 
         lw_top3.loc[1]['effectiveShoot'], lw_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=lw_topSp[1]
))

fig.add_trace(go.Scatterpolar(
      r=[lw_top3.loc[2]['assist'], lw_top3.loc[2]['block'], 
         lw_top3.loc[2]['effectiveShoot'], lw_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=lw_topSp[2]
))



fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 0.8]
    )),
  showlegend=True,
    title = {
        'text': '주요 윙어 능력치 비교',
        'x': 0.46, 
        'y': 0.9
    }
)

fig.show()

### references
- https://plot.ly/python/radar-chart/
- https://wooiljeong.github.io/python/python_plotly/
- https://www.kaggle.com/gurarako/plotly-radar-chart-pokemon-part-3
- https://bbs.ruliweb.com/family/4749/board/100051/read/9488903